# Dask local cluster example

## What is Dask? (https://docs.dask.org/en/latest/)

* combine a blocked algorithm approach
* with dynamic and memory aware task scheduling
* to realise a parallel out-of-core NumPy clone
* optimized for interactive computational workloads

## Monte-Carlo estimate with Dask on multiple CPUs

We define a Dask cluster with 8 CPUs and 24 GB of memory.

In [1]:
import dask.distributed

In [2]:
cluster = dask.distributed.LocalCluster(
    n_workers=1, threads_per_worker=8, memory_limit=24e9,
    ip="0.0.0.0"
)

client = dask.distributed.Client(cluster)
client

<Client: 'tcp://10.0.4.100:37743' processes=1 threads=8, memory=24.00 GB>

### Use dask.array for randomly chosen positions

In [3]:
import numpy, dask.array

In [4]:
def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_shape[0] / number_of_chunks), 2)
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    display(xy)
    
    return pi

### Let's calculate again...
Observe the wall time decreases of the 1 Gigabyte and 10 Gigabyte random sample $\pi$ estimates!

In [5]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000, number_of_chunks=10) # 1 GB


from 1.0 GB randomly chosen positions
   pi estimate: 3.141691136
   pi error: 9.84824102068238e-05



dask.array<uniform, shape=(62500000, 2), dtype=float64, chunksize=(6250000, 2), chunktype=numpy.ndarray>

CPU times: user 48.6 ms, sys: 5.39 ms, total: 54 ms
Wall time: 655 ms


In [6]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1416248384
   pi error: 3.21848102067257e-05



dask.array<uniform, shape=(625000000, 2), dtype=float64, chunksize=(6250000, 2), chunktype=numpy.ndarray>

CPU times: user 551 ms, sys: 41.2 ms, total: 593 ms
Wall time: 3.72 s


### Let's go larger than memory...
Because Dask splits the computation into single managable tasks, we can scale up easily!

In [7]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=250) # 100 GB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14161324224
   pi error: 2.0588650206931902e-05



dask.array<uniform, shape=(6250000000, 2), dtype=float64, chunksize=(25000000, 2), chunktype=numpy.ndarray>

CPU times: user 2.98 s, sys: 174 ms, total: 3.15 s
Wall time: 34.2 s


### Are we now better than single precision floating point resolution?
Not at all, if we require an order of magnitude better...

In [8]:
numpy.finfo(numpy.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

## We could increase the local cluster CPU resources...
However, the above Dask cluster size is always limited by the memory/CPU resources of a single compute node.

In [1]:
# %time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_500) # 1 TB

In [2]:
client.close()
cluster.close()

NameError: name 'client' is not defined